In [3]:
# imports

import os
import pandas as pd
import numpy as np
import scipy.io
from pathlib import Path
import cv2


In [ ]:
# Define dataset directory
dataset_dir = "./datasets"

# Function to automatically detect delimiter in CSV files
def detect_delimiter(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        first_line = file.readline()
        if ',' in first_line:
            return ','
        elif ';' in first_line:
            return ';'
        elif '\t' in first_line:
            return '\t'
        else:
            return None  # Default to pandas auto-detection

# Load all CSV files in the datasets directory
csv_files = [f for f in os.listdir(dataset_dir) if f.endswith('.csv')]
dataframes = {}

for csv_file in csv_files:
    file_path = os.path.join(dataset_dir, csv_file)
    delimiter = detect_delimiter(file_path)
    
    try:
        # Special handling for fullspecs.csv
        if csv_file == 'fullspecs.csv':
            # Read the CSV without setting the first row as headers
            df = pd.read_csv(file_path, delimiter=delimiter, header=None)
            
            # Transpose the dataframe to switch rows and columns
            df = df.transpose()
            
            # Set the first row as the column headers
            df.columns = df.iloc[0]
            
            # Drop the first row as it is now the header
            df = df[1:]
            
            # Shuffle and reset index
            df = df.sample(frac=1, random_state=42).reset_index(drop=True)
        else:
            # Try with detected delimiter
            df = pd.read_csv(file_path, delimiter=delimiter)
        
        dataframes[csv_file] = df
        print(f"Successfully loaded {csv_file} with delimiter '{delimiter}'")
    except:
        # If it fails, let pandas try to figure it out
        df = pd.read_csv(file_path, sep=None, engine='python')
        dataframes[csv_file] = df
        print(f"Successfully loaded {csv_file} with auto-detected delimiter")

# Display information about each dataframe
for name, df in dataframes.items():
    print(f"\n=== {name} ===")
    print(f"Shape: {df.shape}")
    print(f"Columns: {df.columns.tolist()}")
    print("Sample data:")
    print(df.head(3))

In [57]:
# Extract the first 14 columns, 'Fuel System', and 'Trans Description Cont.' for fullspecs.csv
columns_to_extract = list(dataframes['fullspecs.csv'].columns[:14]) + ['Fuel System', 'Trans Description Cont.']
dataframes['fullspecs.csv'] = dataframes['fullspecs.csv'][columns_to_extract]

columns_to_drop = ['Transmission', 'Base Curb Weight (lbs)', 'Front Hip Room (in)']
dataframes['fullspecs.csv'] = dataframes['fullspecs.csv'].drop(columns=columns_to_drop, errors='ignore')
dataframes['fullspecs.csv'].rename(columns={'Trans Description Cont.': 'Transmission'}, inplace=True)


#first_col = dataframes['fullspecs.csv'].columns[0]
# Rename using the actual column name (which is likely np.nan, not the string 'NaN')
dataframes['fullspecs.csv'] = dataframes['fullspecs.csv'].rename(columns={dataframes['fullspecs.csv'].columns[0]: 'Car Description'})


# Verify the updated dataframe
dataframes['fullspecs.csv'].head()

,Car Description,MSRP,Gas Mileage,Engine,EPA Class,Style Name,Drivetrain,Passenger Capacity,Passenger Doors,Body Style,EPA Classification,Fuel System,Transmission
0,2019 Lexus RC Specs: RC 300 F SPORT RWD,"$47,860",21 mpg City/30 mpg Hwy,"Intercooled Turbo Premium Unleaded I-4, 2.0 L",Subcompact Cars,F SPORT RWD,Rear Wheel Drive,4,2,2dr Car,Subcompact Cars,Port/Direct Injection,Automatic w/OD
1,2007 Chrysler 300 Specs: 4-Door Sedan 300 RWD,"$24,445",21 mpg City/28 mpg Hwy,"Gas V6, 2.7L",Large,4dr Sdn 300 RWD,Rear Wheel Drive,5,4,4dr Car,Large,MPI,Automatic
2,2010 Toyota Sequoia Specs: RWD LV8 6-Speed AT Ltd,"$49,140",14 mpg City/19 mpg Hwy,"Gas V8, 5.7L",2WD Sport Utility Vehicle,RWD LV8 6-Spd AT Ltd (Natl),Rear Wheel Drive,8,4,Sport Utility,2WD Sport Utility Vehicle,Electronic Fuel Injection,Automatic
3,2014 INFINITI Q50 Specs: 4-Door Sedan AWD,"$38,950",19 mpg City/27 mpg Hwy,"Premium Unleaded V-6, 3.7 L",Mid-Size Cars,AWD,All Wheel Drive,5,4,4dr Car,Mid-Size Cars,Sequential MPI,Automatic w/OD
4,2013 Toyota Yaris Specs: 5-Door Liftback Autom...,"$16,430",30 mpg City/36 mpg Hwy,"Gas I4, 1.5L",Subcompact Cars,5dr Liftback Auto LE (Natl),Front Wheel Drive,5,4,4dr Car,Subcompact Cars,Electronic Fuel Injection,Automatic


In [58]:
#dataframes['all-vehicles-model.csv'].columns

dataframes['all-vehicles-model.csv']['Fuel Type1'].unique()

array(['Regular Gasoline', 'Diesel', 'Premium Gasoline', 'Natural Gas',
       'Midgrade Gasoline', 'Electricity', 'Hydrogen'], dtype=object)

In [59]:
def combine_car_dataframes(dataframes):
    """Combine all car-related dataframes into a single standardized dataframe"""
    # Create an empty list to hold standardized dataframes
    standardized_dfs = []
    
    # Process used-car-with-prices.csv
    # if 'used-car-with-prices.csv' in dataframes:
    #     df = dataframes['used-car-with-prices.csv'].copy()
    #     # Rename columns to standardized format
    #     df.rename(columns={
    #         'Brand': 'Make',
    #     }, inplace=True)
    #     # Select relevant columns and add N/A price column
    #     df = df[['Make', 'Model', 'Year', 'Fuel', 'Transmission']].copy()
    #     df['Price'] = np.nan  # Add empty price column with N/A values
    #     standardized_dfs.append(df)
        
    # Process used-car-with-prices.csv
    if 'used-car-with-prices.csv' in dataframes:
        df = dataframes['used-car-with-prices.csv'].copy()
        
        # Rename columns to standardized format
        df.rename(columns={
            'Brand': 'Make'
        }, inplace=True)
        
        # Clean model names to remove duplicate make information
        def clean_model_name(row):
            make = str(row['Make']).strip()
            model = str(row['Model']).strip()
            
            # Remove make name from the beginning of model
            if model.startswith(make):
                model = model[len(make):].strip()
            
            # Handle special cases where the model still has the make name repeated
            model_parts = model.split()
            if model_parts and model_parts[0] == make:
                model = ' '.join(model_parts[1:])
                
            return model
        
        # Apply the cleaning function
        df['Model'] = df.apply(clean_model_name, axis=1)
        
        # Select relevant columns
        df = df[['Make', 'Model', 'Year', 'Fuel', 'Transmission']].copy()
        df['Price'] = np.nan  # Add empty price column with N/A values
        standardized_dfs.append(df)
    
    # Process all-vehicles-model.csv
    # if 'all-vehicles-model.csv' in dataframes:
    #     df = dataframes['all-vehicles-model.csv'].copy()
    #     # Rename 'Fuel Type' to 'Fuel' and select relevant columns
    #     df.rename(columns={'Fuel Type': 'Fuel'}, inplace=True)
        
    #     # Standardize transmission values to only Automatic or Manual
    #     def standardize_transmission(trans):
    #         trans = str(trans).lower()
    #         if 'automatic' in trans:
    #             return 'Automatic'
    #         elif 'manual' in trans:
    #             return 'Manual'
    #         else:
    #             return np.nan
        
    #     # Apply standardization to transmission column
    #     df['Transmission'] = df['Transmission'].apply(standardize_transmission)
        
    #     # Select relevant columns
    #     df = df[['Make', 'Model', 'Year', 'Fuel', 'Transmission']].copy()
    #     df['Price'] = np.nan  # Add empty price column with N/A values
    #     standardized_dfs.append(df)
    #     # Process all-vehicles-model.csv
    if 'all-vehicles-model.csv' in dataframes:
        df = dataframes['all-vehicles-model.csv'].copy()
        
        # Standardize fuel types from 'Fuel Type1' column
        def standardize_fuel(fuel):
            fuel = str(fuel).lower()
            if 'gas' in fuel:
                return 'Gas'
            elif fuel == 'diesel':
                return 'Diesel'
            elif fuel == 'electricity':
                return 'Electric'
            elif fuel == 'hydrogen':
                return 'Hydrogen'
            else:
                return 'Unknown'
        
        # Apply fuel standardization
        df['Fuel'] = df['Fuel Type1'].apply(standardize_fuel)
        
        # Standardize transmission values to only Automatic or Manual
        def standardize_transmission(trans):
            trans = str(trans).lower()
            if 'automatic' in trans:
                return 'Automatic'
            elif 'manual' in trans:
                return 'Manual'
            else:
                return np.nan
        
        # Apply standardization to transmission column
        df['Transmission'] = df['Transmission'].apply(standardize_transmission)
        
        # Select relevant columns
        df = df[['Make', 'Model', 'Year', 'Fuel', 'Transmission']].copy()
        df['Price'] = np.nan  # Add empty price column with N/A values
        standardized_dfs.append(df)
        
        
        # Process fullspecs.csv - source of price data
    if 'fullspecs.csv' in dataframes:
        import re
        
        df_specs = dataframes['fullspecs.csv'].copy()
        
        # Verify fullspecs.csv has the expected columns
        print(f"fullspecs.csv columns: {df_specs.columns.tolist()}")
        
        # Extract make, model, and year from Car Description column using regex
        def extract_car_info(desc):
            desc = str(desc).strip()
            
            # First, try to extract the year (first 4 digits)
            year_match = re.search(r'^(\d{4})', desc)
            if not year_match:
                return pd.Series({'Year': np.nan, 'Make': np.nan, 'Model': np.nan})
            
            year = int(year_match.group(1))
            
            # Remove the year from the string
            remaining = desc[4:].strip()
            
            # Split by "Specs:" to get just the make/model part
            parts = remaining.split("Specs:", 1)[0].strip()
            
            # Split the make/model part into words
            words = parts.split()
            
            if len(words) < 1:
                return pd.Series({'Year': year, 'Make': np.nan, 'Model': np.nan})
            
            # First word is the make
            make = words[0]
            
            # Rest is the model (if any)
            model = " ".join(words[1:]) if len(words) > 1 else ""
            
            return pd.Series({
                'Year': year, 
                'Make': make, 
                'Model': model
            })
        
        # Apply extraction to whole column at once
        car_info = df_specs['Car Description'].apply(extract_car_info)
        
        # Create new dataframe with extracted info
        specs_df = pd.DataFrame({
            'Make': car_info['Make'],
            'Model': car_info['Model'],
            'Year': car_info['Year']
        })
        
        # Verify extraction worked properly
        print(f"Extraction results - Make: {specs_df['Make'].nunique()} unique makes, "
            f"Model: {specs_df['Model'].nunique()} unique models, "
            f"Year: {specs_df['Year'].nunique()} unique years")
            
        # Add fuel type based on engine info
        def categorize_fuel(engine):
            engine = str(engine)
            if re.search(r'\bGas\b|\bUnleaded\b', engine, re.IGNORECASE):
                return "Gas"
            elif re.search(r'\bDiesel\b', engine, re.IGNORECASE):
                return "Diesel"
            elif re.search(r'\bElectric\b', engine, re.IGNORECASE):
                return "Electric" 
            elif re.search(r'\bGas/Electric\b|\bHybrid\b', engine, re.IGNORECASE):
                return "Hybrid"
            else:
                return "Unknown"
                
        # Apply fuel categorization to whole column
        specs_df['Fuel'] = df_specs['Engine'].apply(categorize_fuel)
        
        # Standardize transmission values like we did for all-vehicles-model.csv
        def standardize_transmission(trans):
            trans = str(trans).lower()
            if 'automatic' in trans:
                return 'Automatic'
            elif 'manual' in trans:
                return 'Manual'
            else:
                return np.nan
        
        # Add standardized transmission info
        specs_df['Transmission'] = df_specs['Transmission'].apply(standardize_transmission)
        
        # Process MSRP values for the entire column at once
        def clean_price(price):
            if pd.isna(price):
                return np.nan
            price = str(price).replace('"', '').replace('$', '').replace(',', '')
            try:
                return float(price)
            except ValueError:
                return np.nan
                
        specs_df['Price'] = df_specs['MSRP'].apply(clean_price)
        
        # Check if we have any valid price data
        price_count = specs_df['Price'].notna().sum()
        print(f"Extracted {price_count} valid prices from MSRP column")
        
        # Drop rows with missing key data
        specs_df = specs_df.dropna(subset=['Make', 'Model', 'Year'])
        
        # Add to standardized dataframes
        if not specs_df.empty:
            standardized_dfs.append(specs_df)
            print(f"Added {len(specs_df)} cars with price information from fullspecs.csv")
        else:
            print("Warning: No valid data extracted from fullspecs.csv")
    


    # Combine all dataframes with debug info
    
    # if standardized_dfs:
    #     for i, df in enumerate(standardized_dfs):
    #         print(f"DataFrame {i} shape: {df.shape}, columns: {df.columns.tolist()}")
        
    #     combined_df = pd.concat(standardized_dfs, ignore_index=True)
    #     print(f"After concat shape: {combined_df.shape}")
        
    #     # Remove duplicates
    #     before_dedup = len(combined_df)
    #     combined_df = combined_df.drop_duplicates(subset=['Make', 'Model', 'Year'])
    #     print(f"Removed {before_dedup - len(combined_df)} duplicate entries")
        
    #     return combined_df
    # else:
    #     return pd.DataFrame(columns=['Make', 'Model', 'Year', 'Fuel', 'Transmission', 'Price'])
    if standardized_dfs:
        for i, df in enumerate(standardized_dfs):
            # Label the source of each dataframe to track priority
            df['data_source'] = i
            print(f"DataFrame {i} shape: {df.shape}, columns: {df.columns.tolist()}")
        
        # First concatenate all dataframes
        combined_all = pd.concat(standardized_dfs, ignore_index=True)
        print(f"After concat shape: {combined_all.shape}")
        
        # Order the data by source, with fullspecs.csv (typically index 2) being last
        # This makes it override others when we drop duplicates
        combined_all = combined_all.sort_values('data_source')
        
        # Find duplicates but keep the last occurrence (which will be from fullspecs)
        before_dedup = len(combined_all)
        combined_df = combined_all.drop_duplicates(subset=['Make', 'Model', 'Year'], keep='last')
        print(f"Removed {before_dedup - len(combined_df)} duplicate entries")
        
        # Clean up by removing the data_source column
        combined_df = combined_df.drop(columns=['data_source'])
        
        # An extra check to confirm price prioritization
        print(f"Final price data count: {combined_df['Price'].notna().sum()}")
        
        return combined_df
    else:
        return pd.DataFrame(columns=['Make', 'Model', 'Year', 'Fuel', 'Transmission', 'Price'])

# Create combined dataframe
combined_cars_df = combine_car_dataframes(dataframes)

# Show info about the combined dataframe
print(f"\nCombined dataframe shape: {combined_cars_df.shape}")
print("Sample data:")
print(combined_cars_df.head(10))

# Print statistics about Price column
print("\nPrice data statistics:")
price_count = combined_cars_df['Price'].notna().sum()
# Print statistics about Price column
print("\nTransmission data statistics:")
transmission_count = combined_cars_df['Transmission'].notna().sum()
print(f"Number of entries with price data: {price_count}")
print(f"Number of entries with transmission data: {transmission_count}")
print(f"Percentage of entries with price data: {price_count/len(combined_cars_df)*100:.2f}%")
if price_count > 0:
    print(f"Price range: ${combined_cars_df['Price'].min():,.2f} to ${combined_cars_df['Price'].max():,.2f}")
    print(f"Average price: ${combined_cars_df['Price'].mean():,.2f}")

fullspecs.csv columns: ['Car Description', 'MSRP', 'Gas Mileage', 'Engine', 'EPA Class', 'Style Name', 'Drivetrain', 'Passenger Capacity', 'Passenger Doors', 'Body Style', 'EPA Classification', 'Fuel System', 'Transmission']
Extraction results - Make: 43 unique makes, Model: 426 unique models, Year: 29 unique years
Extracted 32262 valid prices from MSRP column
Added 32316 cars with price information from fullspecs.csv
DataFrame 0 shape: (4340, 7), columns: ['Make', 'Model', 'Year', 'Fuel', 'Transmission', 'Price', 'data_source']
DataFrame 1 shape: (47523, 7), columns: ['Make', 'Model', 'Year', 'Fuel', 'Transmission', 'Price', 'data_source']
DataFrame 2 shape: (32316, 7), columns: ['Make', 'Model', 'Year', 'Fuel', 'Transmission', 'Price', 'data_source']
After concat shape: (84179, 7)
Removed 55184 duplicate entries
Final price data count: 3850

Combined dataframe shape: (28995, 6)
Sample data:
            Make                        Model  Year    Fuel Transmission  \
2885   Chevrolet  

Cleaning data

In [60]:
# Show the results
print("Sample of final dataframe:")
# Sort the final dataframe by Price in descending order and display the top 10 rows
sorted_final_df = combined_cars_df.sort_values(by='Price', ascending=True)
print(sorted_final_df.head(25))
print(sorted_final_df.isna().sum())

Sample of final dataframe:
             Make     Model  Year     Fuel Transmission    Price
60568  Mitsubishi    Mirage  1990  Unknown          NaN   7839.0
78960     Hyundai    Accent  1995  Unknown          NaN   8079.0
63700     Hyundai    Accent  1996  Unknown          NaN   8455.0
63632      Nissan    Sentra  1990  Unknown          NaN   8649.0
62600  Mitsubishi    Mirage  1991  Unknown          NaN   8829.0
67337         Kia       Rio  2001      Gas       Manual   8895.0
63203  Volkswagen      Golf  1990  Unknown          NaN   8995.0
67196      Nissan    Sentra  1993  Unknown          NaN   8995.0
59968      Nissan    Sentra  1991  Unknown          NaN   9265.0
54450  Volkswagen      Golf  1991  Unknown          NaN   9270.0
66194        Ford   Mustang  1990  Unknown          NaN   9456.0
67217     Hyundai   Elantra  1992  Unknown          NaN   9645.0
82916     Hyundai    Accent  2000      Gas       Manual   9699.0
61278  Mitsubishi    Mirage  1992  Unknown          NaN   9779.

Double-check of the cleaned up data

In [61]:
# Print statistics about Price column
print("\nPrice data statistics:")
price_count = sorted_final_df['Price'].notna().sum()
# Print statistics about Price column
print("\nTransmission data statistics:")
transmission_count = sorted_final_df['Transmission'].notna().sum()
print(f"Number of entries with price data: {price_count}")
print(f"Number of entries with transmission data: {transmission_count}")
print(f"Percentage of entries with price data: {price_count/len(sorted_final_df)*100:.2f}%")
if price_count > 0:
    print(f"Price range: ${sorted_final_df['Price'].min():,.2f} to ${sorted_final_df['Price'].max():,.2f}")
    print(f"Average price: ${sorted_final_df['Price'].mean():,.2f}")


Price data statistics:

Transmission data statistics:
Number of entries with price data: 3850
Number of entries with transmission data: 28399
Percentage of entries with price data: 13.28%
Price range: $7,839.00 to $548,800.00
Average price: $47,009.17


Exporting the DF into a CSV-file

In [62]:
# Export the combined dataframe to a CSV file
output_dir = Path("./processed_data")
output_dir.mkdir(exist_ok=True)  # Create the directory if it doesn't exist

# Save the combined dataframe to CSV
output_path = output_dir / "combined_cars.csv"
combined_cars_df.to_csv(output_path, index=False)
print(f"Combined car data saved to {output_path}")

# Display the first few rows of the exported file to verify
try:
    verification_df = pd.read_csv(output_path)
    print("\nVerification of exported CSV:")
    print(f"Shape: {verification_df.shape}")
    print(verification_df.head(3))
except Exception as e:
    print(f"Error verifying the exported CSV: {e}")

Combined car data saved to processed_data\combined_cars.csv

Verification of exported CSV:
Shape: (28995, 6)
        Make                       Model  Year    Fuel Transmission  Price
0  Chevrolet                     Beat LT  2015  Petrol       Manual    NaN
1       Ford  Figo 1.2P Titanium Plus MT  2012  Petrol       Manual    NaN
2    Hyundai     Grand i10 1.2 Kappa Era  2015  Petrol       Manual    NaN


Finding prices of used cars online is either done manually or well, to make our lives easier: data/web scraping. Sadly, most APIs regarding car prices aren't free and it's hard to get the data without getting into legal trouble. So I will only use the pricing data that I currently have from the CSV and mainly focus on the image classification of the cars.

____

I have found out that I didn't implement all car classes and to add more challenge to it, I will implement the missing ones to the existing model(s).

In [ ]:
all_brands_df = pd.read_csv("./datasets/brands.csv", delimiter=',', header=0)
all_brands_df.head()


,id,url_hash,url,name,logo,created_at,updated_at,Unnamed: 7
0,1,78383652,https://www.autoevolution.com/ac/,AC,https://s1.cdn.autoevolution.com/images/produc...,NaN,2024-10-23T08:32:05.000000Z,2024-10-23T08:32:05.000000Z
1,2,6a1df75c,https://www.autoevolution.com/acura/,ACURA,https://s1.cdn.autoevolution.com/images/produc...,NaN,2024-10-23T08:32:05.000000Z,2024-10-23T08:32:05.000000Z
2,3,0608c25a,https://www.autoevolution.com/alfa-romeo/,ALFA ROMEO,https://s1.cdn.autoevolution.com/images/produc...,NaN,2024-10-23T08:32:05.000000Z,2024-10-23T08:32:05.000000Z
3,4,a829c754,https://www.autoevolution.com/alpine/,ALPINE,https://s1.cdn.autoevolution.com/images/produc...,NaN,2024-10-23T08:32:05.000000Z,2024-10-23T08:32:05.000000Z
4,5,85a1d7e1,https://www.autoevolution.com/ariel/,ARIEL,https://s1.cdn.autoevolution.com/images/produc...,NaN,2024-10-23T08:32:05.000000Z,2024-10-23T08:32:05.000000Z
5,6,c4aba20b,https://www.autoevolution.com/aro/,ARO,https://s1.cdn.autoevolution.com/images/produc...,NaN,2024-10-23T08:32:05.000000Z,2024-10-23T08:32:05.000000Z
6,7,d519e48e,https://www.autoevolution.com/artega/,ARTEGA,https://s1.cdn.autoevolution.com/images/produc...,NaN,2024-10-23T08:32:05.000000Z,2024-10-23T08:32:05.000000Z
7,8,bf4a9110,https://www.autoevolution.com/aston-martin/,ASTON MARTIN,https://s1.cdn.autoevolution.com/images/produc...,NaN,2024-10-23T08:32:05.000000Z,2024-10-23T08:32:05.000000Z
8,9,47f8c52c,https://www.autoevolution.com/audi/,AUDI,https://s1.cdn.autoevolution.com/images/produc...,NaN,2024-10-23T08:32:05.000000Z,2024-10-23T08:32:05.000000Z
9,10,1de3e1c4,https://www.autoevolution.com/aurus/,AURUS,https://s1.cdn.autoevolution.com/images/produc...,NaN,2024-10-23T08:32:05.000000Z,2024-10-23T08:32:05.000000Z


In [7]:
# Extract just the "name" column
car_brands_df = all_brands_df[["name"]].copy()

# Save to processed_data directory
output_dir = Path("./processed_data")
output_dir.mkdir(exist_ok=True)  # Create the directory if it doesn't exist

# Save to CSV
output_path = output_dir / "all_car_brands.csv"
car_brands_df.to_csv(output_path, index=False)

print(f"Car brands saved to {output_path}")

# Verify the exported file
verification_df = pd.read_csv(output_path)
print("\nVerification of exported CSV:")
print(f"Shape: {verification_df.shape}")
print(verification_df.head())

Car brands saved to processed_data\all_car_brands.csv

Verification of exported CSV:
Shape: (124, 1)
         name
0          AC
1       ACURA
2  ALFA ROMEO
3      ALPINE
4       ARIEL
